In [1]:
# The story telling algorithm with distributed similarity mechanism and diffusion model for all documents in the dataset
# Author: Alireza Nouri
# email: apashamoham@miners.utep.edu
# date 12/16/2023

In [1]:
#import packages
from transformers import BertTokenizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
from sklearn.manifold import TSNE

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset

import matplotlib.pyplot as plt

import sys
sys.path.append('..')
from dataset_reader import dataset_loader
from contextual_embedding.contextual_embedding import bert_embedding

import seaborn as sns
import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm
import copy
import random 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import tensorflow as tf
from tensorflow.keras import layers, Model
import nltk

2023-12-16 11:48:41.434249: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-16 11:48:41.434280: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-16 11:48:41.434723: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-16 11:48:41.484158: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
# Load dataset
#read csv file
path = '../../../Dataset/NYTimes/AHNAF_NYTimes/2022.csv'
df = pd.read_csv(path)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(df.columns)
# print(df.shape)
df = df[df['section_name'].isin(['U.S.', 'World'])]
#select the columns
df_selected = df[['pub_date', 'abstract', 'headline', 'article', 'keywords', 'section_name', 'subsection_name', 'web_url']]

# drop none values elements
# print(df_selected.shape)
# df_selected.dropna(axis=0, how='any')
# print(df_selected.shape)

def cleaning_text(text):
    return str(text).replace('##', ' ')

#convert dataframe into a list of dictionaries, each one has all information and id
original_data = []
columns = df_selected.columns

for ind, row in df_selected.iterrows():
    try:
        temp_dict = {}
        temp_dict['id'] = ind

        for el in columns:
            temp_dict[el] = row[el]
        temp_dict['article'] = cleaning_text(temp_dict['article'])
        original_data.append(temp_dict)
    except:
        continue
    


In [4]:

# nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

In [5]:
from copy import deepcopy, copy
import warnings
warnings.filterwarnings("ignore")

MAX_TOKENS_LENGTH = 400
MIN_TOKENS_LENGTH = 100

In [8]:
# pick random samples from dataset
n_samples = len(original_data)
original_data = random.sample(original_data, n_samples)

# eliminate data with large size of text

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

dataset = []
for data in tqdm(original_data):
    try:
        if data['article'] != 'nan':
            z = tokenizer.tokenize(data['article'])
            if (len(z) > MIN_TOKENS_LENGTH) and (len(z) < MAX_TOKENS_LENGTH):
                dataset.append(data)
    except:
        continue
        
# pickle.dump(dataset, open('25_dataset.p', 'wb'))
# dataset = pickle.load(open('25_dataset.p', 'rb'))

# generate document embedding by BERT
context_em = bert_embedding()
contextual_bert_embedding = []
for doc in tqdm(dataset):
    contextual_bert_embedding.append([doc['id'], context_em.embedding_generator(doc['article'], embedding_type='summing')[1][0][1]])

contextual_doc_embedding =  deepcopy(contextual_bert_embedding)   
# contextual_doc_embedding[id, bert_cls_embedding]    


100%|██████████| 8338/8338 [09:59<00:00, 13.91it/s]


In [ ]:
# pickle.dump(contextual_doc_embedding, open('25_contextual_doc_embedding.p', 'wb'))
# pickle.dump(dataset, open('25_dataset.p', 'wb'))
# contextual_doc_embedding = pickle.load(open('25_contextual_doc_embedding.p', 'rb'))
# dataset = pickle.load(open('25_dataset.p', 'rb'))

In [8]:
def resize_embedding_vector(embedding_matrix, input_dim, output_dim, original_embedding_matrix=contextual_doc_embedding, epochs_n=32):
    compressed_dim = output_dim
    # Encoder model
    input_vec = layers.Input(shape=(input_dim,))
    encoded = layers.Dense(compressed_dim, activation='relu')(input_vec)
    
    # Decoder model
    decoded = layers.Dense(input_dim, activation='sigmoid')(encoded)
    
    # Autoencoder
    autoencoder = Model(input_vec, decoded)
    
    # Encoder - for compressing the input
    encoder = Model(input_vec, encoded)
    
    # Decoder - for reconstructing the input from compressed form
    encoded_input = layers.Input(shape=(compressed_dim,))
    decoder_layer = autoencoder.layers[-1]
    decoder = Model(encoded_input, decoder_layer(encoded_input))
    
    # Compile the model
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    
    # # Model summary
    # autoencoder.summary()
    
    # Train the model
    autoencoder.fit(embedding_matrix, embedding_matrix, epochs=epochs_n, batch_size=64, shuffle=True)
    temp = encoder.predict(embedding_matrix)
    temp_dict = dict()
    for id, vec in zip(original_embedding_matrix, temp):
        temp_dict[id[0]] = vec
        
    return temp_dict

In [7]:
# pickle.dump(contextual_doc_embedding, open('25_contextual_doc_embedding.p', 'wb'))
# pickle.dump(dataset, open('25_dataset.p', 'wb'))
# contextual_doc_embedding = pickle.load(open('25_contextual_doc_embedding.p', 'rb'))
# dataset = pickle.load(open('25_dataset.p', 'rb'))

In [ ]:
# # print(type(contextual_doc_embedding))
# # print(type(contextual_bert_embedding))
# # print(type(contextual_doc_embedding[21517]))
# # print(type(contextual_bert_embedding[0][1]))
# # new_context = []
# # for k in contextual_doc_embedding:
# #     new_context.append([k, torch.tensor(contextual_doc_embedding[k])])
# contextual_doc_embedding = new_context

In [ ]:
title_id_dict = dict()
for data in dataset:
    title_id_dict[data['id']] = data['headline']

abstract_id_dict = dict()
for data in dataset:
    abstract_id_dict[data['id']] = data['abstract']

text_id_dict = dict()
for data in dataset:
    text_id_dict[data['id']] = data['article']

date_id_dict = dict()
for data in dataset:
    date_id_dict[data['id']] = data['pub_date']

In [ ]:
# pickle.dump(title_id_dict, open('25_title_id_dict.p', 'wb'))
# pickle.dump(abstract_id_dict, open('25_abstract_id_dict.p', 'wb'))
# pickle.dump(text_id_dict, open('25_text_id_dict.p', 'wb'))
# pickle.dump(date_id_dict, open('25_date_id_dict.p', 'wb'))
# pickle.dump(tf_idf_dict, open('25_tf_idf_dict.p', 'wb'))
# pickle.dump(data_timestamps, open('25_data_timestamps.p', 'wb'))
# pickle.dump(contextual_doc_embedding, open('25_contextual_doc_embedding.p', 'wb'))
# pickle.dump(contextual_bert_embedding, open('25_contextual_bert_embedding.p', 'wb'))
# pickle.dump(contextual_dict, open('25_contextual_dict.p','wb'))


In [10]:
####################Reduce the size of number of documnets
embedding_col = [w[1].tolist() for w in contextual_doc_embedding]
contextual_doc_embedding = resize_embedding_vector(embedding_col, len(embedding_col[0]), 128, contextual_doc_embedding)
new_context = []
for k in contextual_doc_embedding:
    new_context.append([k, torch.tensor(contextual_doc_embedding[k])])
contextual_doc_embedding = new_context

contextual_dict = dict()
for data_ in contextual_doc_embedding:
    contextual_dict[data_[0]] = data_[1]

2023-12-16 11:49:34.430108: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-16 11:49:34.430390: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-16 11:49:34.443962: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/32


2023-12-16 11:49:56.210512: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb7d42795d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-16 11:49:56.210530: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2023-12-16 11:49:56.210535: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA RTX A6000, Compute Capability 8.6
2023-12-16 11:49:56.215588: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-16 11:49:56.250595: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-12-16 11:49:56.318231: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


131/131 [==============================] - 1s 2ms/step - loss: 3.9830
Epoch 2/32
131/131 [==============================] - 0s 1ms/step - loss: 3.8902
Epoch 3/32
131/131 [==============================] - 0s 1ms/step - loss: 3.8669
Epoch 4/32
131/131 [==============================] - 0s 1ms/step - loss: 3.8518
Epoch 5/32
131/131 [==============================] - 0s 1ms/step - loss: 3.8410
Epoch 6/32
131/131 [==============================] - 0s 1ms/step - loss: 3.8327
Epoch 7/32
131/131 [==============================] - 0s 1ms/step - loss: 3.8266
Epoch 8/32
131/131 [==============================] - 0s 1ms/step - loss: 3.8221
Epoch 9/32
131/131 [==============================] - 0s 1ms/step - loss: 3.8187
Epoch 10/32
131/131 [==============================] - 0s 1ms/step - loss: 3.8157
Epoch 11/32
131/131 [==============================] - 0s 1ms/step - loss: 3.8131
Epoch 12/32
131/131 [==============================] - 0s 1ms/step - loss: 3.8107
Epoch 13/32
131/131 [===============

In [11]:
# pickle.dump(contextual_doc_embedding, open('25_contextual_doc_embedding.p', 'wb'))
# pickle.dump(dataset, open('25_dataset.p', 'wb'))

In [11]:
# Create the TF-IDF vectorizer
vectorizer_tfidf = TfidfVectorizer(stop_words=stopwords)

# Fit the vectorizer to the documents and transform the documents into their TF-IDF representation
tfidf_matrix = vectorizer_tfidf.fit_transform([w['article'] for w in dataset])
list_of_words_in_tfidf = vectorizer_tfidf.get_feature_names_out()
words_list_tfidf = tfidf_matrix.toarray()

tf_idf_dict = dict()
for data, id in zip(words_list_tfidf, dataset):
    tf_idf_dict[id['id']] = data

In [16]:
# in this section, a model uses document embedding to predict the tf-idf
docemb_tfidf = []
for emb, tf_idf in zip(contextual_doc_embedding, words_list_tfidf):
    docemb_tfidf.append([emb[1], tf_idf])

embeddings_size = len(docemb_tfidf[0][0])
X_embeddings = np.random.rand(len(contextual_doc_embedding), embeddings_size)
X_tfidf = np.random.rand(len(contextual_doc_embedding), len(docemb_tfidf[0][1]))
for i in range(len(X_embeddings)):
    X_embeddings[i] = docemb_tfidf[i][0]

for i in range(len(X_tfidf)):
    X_tfidf[i] = docemb_tfidf[i][1]

# Neural Network
model_tfidf = Sequential([
    Dense(2*embeddings_size, activation='relu', input_shape=(embeddings_size,)),
    # Dense(len(docemb_tfidf[0][1])/4, activation='relu'),
    Dense(len(docemb_tfidf[0][1]), activation='linear')  # Output size is vocabulary size
])

model_tfidf.compile(optimizer='adam', loss='mse')
model_tfidf.summary()

# Train the model
model_tfidf.fit(X_embeddings[:1000], X_tfidf[:1000], epochs=50, batch_size=512)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 256)               33024     
                                                                 
 dense_3 (Dense)             (None, 39808)             10230656  
                                                                 
Total params: 10263680 (39.15 MB)
Trainable params: 10263680 (39.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
2/2 [==============================] - 1s 14ms/step - loss: 0.4568
Epoch 2/50
2/2 [==============================] - 0s 13ms/step - loss: 0.1859
Epoch 3/50
2/2 [==============================] - 0s 21ms/step - loss: 0.1420
Epoch 4/50
2/2 [==============================] - 0s 15ms/step - loss: 0.0820
Epoch 5/50
2/2 [==============================] - 0s 15ms/step - loss: 0.0507
Epoch 6/50
2/2 [====

In [13]:
# collect all timestamps
all_timestamps = set()
for i in dataset:
    all_timestamps.add(i['pub_date'][5:])
sorted_ts = sorted(list(all_timestamps))

timestamp_width = 3
ts_groups = [sorted_ts[timestamp_width*i:timestamp_width*i+3] for i in range(int(len(sorted_ts)/timestamp_width))]
ts_dict = dict()
for i in range(len(ts_groups)):
    ts_dict[i] = ts_groups[i]
    
data_timestamps = dict()
for w in ts_dict.keys():
    data_timestamps[w] = []
    
for d, emb, tf_ in zip(dataset, contextual_doc_embedding, words_list_tfidf):
    for ke, val in ts_dict.items():
        if d['pub_date'][5:] in val:
            data_timestamps[ke].append([d['id'], d['pub_date'][5:], d['article'], emb[1], tf_])

In [14]:
centroids_ts_group = [] # centroids in each timestamp
emb_and_centroids = []

for i in tqdm(data_timestamps):
    emb_groups = [w[3].tolist() for w in data_timestamps[i]]
    N_CLUSTERS = int(len(emb_groups)/3) # this is hardcoding the number of clusters. later on, it must changed to dynamic approaches
    kmeans = KMeans(n_clusters=N_CLUSTERS)
    kmeans.fit(emb_groups)
    labels = kmeans.labels_
    centroids = kmeans.cluster_centers_
    centroids_ts_group.append(centroids)
    temp_list = []
    for ind, emb in zip(labels, emb_groups):
        temp_list.append([emb, centroids[ind]])
    emb_and_centroids.append(temp_list)
        
  #emb_and_centroids : [timestamps][number_of_clusters][0: doc_embedding, 1: centroids]  

100%|██████████| 88/88 [00:20<00:00,  4.39it/s]


In [15]:
emb_and_conditions = []
for i in range(len(data_timestamps)-1) :
    emb_groups = [w[3].tolist() for w in data_timestamps[i]]
    temp_list = []
    for em in emb_groups:
        for cond_set in emb_and_centroids[i+1]:
            temp_list.append([em, cond_set])
    emb_and_conditions.append(temp_list)

data_for_diffusion = []
for t_ in emb_and_conditions:
    for c_ in t_:
        data_for_diffusion.append(c_)
# data_for_diffusion : [[[document_embedding][condition1,condition2]], [[document_embedding][condition1,condition2]]]

In [38]:
# make the training dataset smaller
# data_for_diffusion = data_for_diffusion[:10000]


vec_size = len(contextual_doc_embedding[0][1])
new_data_w_condition = torch.randn((len(data_for_diffusion),3*vec_size))
for i in tqdm(range(len(data_for_diffusion))):
    for j in range(vec_size):
        new_data_w_condition[i][j] = data_for_diffusion[i][0][j]
        new_data_w_condition[i][j+vec_size] = data_for_diffusion[i][1][0][j]
        new_data_w_condition[i][j+2*vec_size] = data_for_diffusion[i][1][1][j]
        
new_data_ = torch.randn((len(data_for_diffusion),len(contextual_doc_embedding[0][1])))
for i in tqdm(range(len(data_for_diffusion))):
    for j in range(len(contextual_doc_embedding[0][1])):
        new_data_[i][j] = data_for_diffusion[i][0][j]

100%|██████████| 10000/10000 [00:03<00:00, 2847.03it/s]


In [19]:
# pickle.dump(new_data_, open('25_new_data_.p', 'wb'))
# pickle.dump(model_tfidf, open('25_model_tfidf.p', 'wb'))
# pickle.dump(new_data_w_condition, open('25_new_data_w_condition.p', 'wb'))
# new_data_ = pickle.load(open('25_new_data_.p', 'rb'))
# model_tfidf = pickle.load(open('25_model_tfidf.p', 'rb'))
# new_data_w_condition = pickle.load(open('25_new_data_w_condition.p', 'rb'))

In [39]:
###############number of stpes calculated separatly in 02_Diffused_based_storytelling file #############
# # Calculating the parameters for the diffusion model

# # calculate the number of steps to add noises into data to feed them into the diffusion model
# noise_factor_list = [0.15]
# data_final = []
# for noise_factor in noise_factor_list:
#     n_steps = 300
#     data_no_noise = torch.randn((len(contextual_doc_embedding),len(contextual_doc_embedding[0][1])))
#     for i in range(len(contextual_doc_embedding)):
#         for j in range(len(contextual_doc_embedding[0][1])):
#             data_no_noise[i][j] = contextual_doc_embedding[i][1][j]
#     data_final.append(data_no_noise)
# #     data_final.append(data_no_noise[:100][:]) this line has been changed to upper line

# diffused_data_steps_final = []
# for i in range(len(data_final)):
#     vec_size = len(contextual_doc_embedding[0][1])

#     diffused_data_steps = [data_final[i]]
#     for _ in range(n_steps):
#         data_final[i] = data_final[i] + noise_factor * torch.randn_like(data_final[i])
#         diffused_data_steps.append(data_final[i])
#     diffused_data_steps_final.append(diffused_data_steps)

# similarity_noise_final = []
# for m in range(len(data_final)):
#     similarity_noise = {}
#     for i in range(n_steps+1):
#         similarity_noise[i] = []


#     for i in tqdm(range(n_steps+1)):
#         temp = []
#         for j in range(len(diffused_data_steps_final[m][0])):
#             temp.append(cosine_similarity([diffused_data_steps_final[m][0][j].tolist()],[diffused_data_steps_final[m][i][j].tolist()])[0][0])
#         similarity_noise[i].append(temp)
    
#     similarity_noise_final.append(similarity_noise)

# # plot for all different noise factors
# for n in range(len(data_final)):
#     res = [np.mean(similarity_noise_final[n][w]) for w in similarity_noise_final[n].keys() if w%100 == 0]
#     plt.plot([100*i for i in range(len(res))],res, '-o')
# plt.show()

# #plot for the last noise factor
# plt.plot([100*i for i in range(len(res))],res, '-o')
# plt.title('Showing how the data and noise combined in different steps')
# plt.xlabel('number of steps')
# plt.ylabel('cosine similarities')
# plt.show()

# n_step_list = []
# c = 0
# for noise_f in similarity_noise_final: 
#     for key, val in noise_f.items():
#         if np.mean(val) < 0.5:
#             print('for noise factor {} the number of step is {}'.format(noise_factor_list[c],key))
#             n_step_list.append(key)
#             break
#     c += 1
    

# noise_factor = noise_factor_list[0]
# n_steps = n_step_list[0]
n_steps = 50
noise_factor = 0.15

In [40]:
len(data_for_diffusion)

10000

In [41]:
# diffusion models
diffused_data_steps = [new_data_w_condition]
for _ in tqdm(range(n_steps)):
    new_data_w_condition[:,:vec_size] += noise_factor * torch.randn_like(new_data_)
    diffused_data_steps.append(new_data_w_condition)
    


100%|██████████| 50/50 [00:00<00:00, 133.11it/s]


In [23]:
# pickle.dump(diffused_data_steps, open('25_diffused_data_steps.p', 'wb'))
# diffused_data_steps = pickle.load(open('25_diffused_data_steps.p', 'rb'))

In [42]:
# Convert the diffused data steps into a dataset
# Each sample is a pair of subsequent diffused data steps
diffused_data_pairs=[torch.stack((diffused_data_steps[i+1],diffused_data_steps[i]),dim= 1) for i in range(n_steps)]
diffused_data = torch.cat(diffused_data_pairs, dim=0)
dataset_ = TensorDataset(*torch.unbind(diffused_data, dim=1))

In [43]:
#  Create a Dataloader to handle batching of dataset
dataloader = DataLoader(dataset_, batch_size=256, shuffle=True)

# Neural Network
model = nn.Sequential(nn.Linear(3*vec_size, vec_size), nn.ReLU(), nn.Dropout(0.2),
                     nn.Linear(vec_size, vec_size), nn.ReLU(), nn.Dropout(0.2),
                      nn.Linear(vec_size, 2*vec_size), nn.ReLU(), nn.Dropout(0.2),
                     nn.Linear(2*vec_size, 3*vec_size)
                     )

# now move the  model to device
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr = 0.001)
loss_func = nn.MSELoss()

In [44]:
# Training loop
for i in range(1, 501):
    model.train()

    for batch_noisy, batch_data in dataloader:
        # Move data to device
        batch_noisy, batch_data = batch_noisy.to(device), batch_data.to(device)

        optimizer.zero_grad()
        batch_denoised = model(batch_noisy)
        loss = loss_func(batch_denoised, batch_data)
        loss.backward()
        optimizer.step()

    if i % 10 == 0:
        print(f'Epoch {i}, Loss: {loss.item()}')

Epoch 10, Loss: 6.820949554443359
Epoch 20, Loss: 7.862959861755371
Epoch 30, Loss: 5.326531887054443
Epoch 40, Loss: 7.327765941619873
Epoch 50, Loss: 7.02612829208374
Epoch 60, Loss: 5.420815944671631
Epoch 70, Loss: 7.1498517990112305
Epoch 80, Loss: 5.689389705657959
Epoch 90, Loss: 6.386630535125732
Epoch 100, Loss: 6.168487548828125
Epoch 110, Loss: 6.018319606781006
Epoch 120, Loss: 6.360808849334717
Epoch 130, Loss: 5.305604934692383
Epoch 140, Loss: 6.055332183837891
Epoch 150, Loss: 6.240146636962891
Epoch 160, Loss: 5.417392253875732
Epoch 170, Loss: 5.3054609298706055
Epoch 180, Loss: 7.627220153808594
Epoch 190, Loss: 7.104910850524902
Epoch 200, Loss: 6.9619245529174805
Epoch 210, Loss: 6.322051048278809
Epoch 220, Loss: 5.991157531738281
Epoch 230, Loss: 7.260288238525391
Epoch 240, Loss: 5.267133712768555
Epoch 250, Loss: 5.832584381103516
Epoch 260, Loss: 4.940969467163086
Epoch 270, Loss: 6.420839309692383
Epoch 280, Loss: 5.463011741638184
Epoch 290, Loss: 4.59209823

In [45]:
# pickle.dump(model, open('25_model_vectorsize128_diffusion.p', 'wb'))
# model = pickle.load(open('25_model_vectorsize128_diffusion.p', 'rb'))

In [47]:
# import sys
# def sizeof_fmt(num, suffix='B'):
#     ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
#     for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
#         if abs(num) < 1024.0:
#             return "%3.1f %s%s" % (num, unit, suffix)
#         num /= 1024.0
#     return "%.1f %s%s" % (num, 'Yi', suffix)

# for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
#                           locals().items())), key= lambda x: -x[1])[:10]:
#     print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

In [48]:
collect_ids = {}
for i in range(len(data_timestamps)):
    collect_ids[i] = []
for t in range(len(data_timestamps)):
    for doc_ in data_timestamps[t]:
        collect_ids[t].append(doc_[0])

In [49]:
def cosin_similarity_doc_emb(vec_one, vec_two):
    return cosine_similarity([vec_one], [vec_two])[0][0]

In [50]:
def keyword_extractor(x_tf_idf_, list_of_words_in_tfidf_, top_keywords_):
    temp_dict = dict()
    for i,j in zip(x_tf_idf_, list_of_words_in_tfidf_):
        temp_dict[j] = i
    return sorted(temp_dict.items(), key=lambda x:x[1], reverse=True)[:top_keywords_]

In [51]:
def merge_keywords(list_one, list_two, factor_merging):
    merge_dict = dict()
    for data_ in list_one:
        merge_dict[data_[0]] = factor_merging * data_[1]
    for data_ in list_two:
        if data_[0] in merge_dict:
            merge_dict[data_[0]] = (merge_dict[data_[0]] + data_[1])/2
        else:
            merge_dict[data_[0]] = data_[1]
    return sorted(merge_dict.items(), key=lambda x:x[1], reverse=True)[:len(list_one)]

In [52]:
def keyword_similarity(list_one, list_two):
    score = 0
    
    for data_ in list_two:
        for data in list_one:
            if data[0] == data_[0]:
                score += min(data[1], data_[1])
    return score

In [53]:
def similarity_evaluator_emb_tf_idf(seed_emb, seed_tf_idf, list_of_emb_generated, list_of_tfidf_generated):
    cosine_sim = []
    tf_idf_sim = []
    for gen_emb in list_of_emb_generated:
        cosine_sim.append(cosine_similarity([seed_emb], [gen_emb])[0][0])
    for gen_tfidf in list_of_tfidf_generated:
        tf_idf_sim.append(sum(np.multiply(gen_tfidf, seed_tf_idf)))
    #normalized tf_idf
    temp_max = max(tf_idf_sim)+ 0.0001
    tf_idf_sim = [w/temp_max for w in tf_idf_sim]
    return [n*m for n,m in zip(cosine_sim, tf_idf_sim)]
        
    

In [54]:
def tf_idf_normalizer(tf_idf_vec):
    temp_list = []
    for el in tf_idf_vec:
        if (el > 0) and (el < 1):
            temp_list.append(el)
        elif el > 1:
            temp_list.append(1.0)
        else:
            temp_list.append(0.0)
    return temp_list

In [55]:
def ts_finder(ts_, collect_ids):
    for ke, val in collect_ids.items():
        if ts_ in val:
            return ke
    return -1

In [56]:
def remove_duplicate_embedding(list_of_embedding, max_sim=0.9):
    res_ = list()
    if len(list_of_embedding) == 2:
        return list_of_embedding
    else:
        res_.append(list_of_embedding[0])
        for i in range(1, len(list_of_embedding)):
            max_ = 0
            for j in range(len(res_)):
                cos_sim = cosine_similarity([res_[j]], [list_of_embedding[i]])[0][0]
                max_ = max_ if cos_sim < max_ else cos_sim
            if max_ < max_sim:
                res_.append(list_of_embedding[i])
    return res_

In [57]:
# this function recieves a seed and generats the relevant story to that seed
def story_gen_emb_tfidf_novel_method(seed_, collect_ids, min_size_generated_data_per_ts=50, min_acceptable_similarity= 0.9, min_keyword_overlapping= 0.60):
    ###SEARCH ALGORITHM
    starting_ts = ts_finder(seed_, collect_ids)
        
    seed_doc = seed_
    storyline = []
    storyline.append(seed_doc)
    storyline_keywords = keyword_extractor(tf_idf_dict[seed_doc], list_of_words_in_tfidf, 100)
    aux_doc = False
    for ts_ids in list(collect_ids.keys())[starting_ts+1:]:
        #preparing conditions and noises for diffusion model
        data_for_generator = []
        data_ids = [w[0] for w in data_timestamps[ts_ids]]
        seed_embedding = contextual_dict[seed_doc]
        temp = list()
        while len(data_for_generator) < min_size_generated_data_per_ts:
            for cent_ in centroids_ts_group[ts_ids]:
                temp[:vec_size] = seed_embedding
                temp[vec_size:] = cent_
                # data_for_generator.append([contextual_dict[seed_doc], cent_])
                data_for_generator.append(temp)
    
        random_doc_tensor = torch.randn((len(data_for_generator), len(data_for_generator[0]))).to(device)
        for i in range(len(data_for_generator)):
            random_doc_tensor[i] = torch.from_numpy(np.array(data_for_generator[i]))
                
        average_data = torch.mean(abs(random_doc_tensor))
        noise_data = torch.randn((len(data_for_generator), vec_size)).to(device)
        average_noise = torch.mean(abs(noise_data))
        noise_coef = average_data/average_noise
        noise_data = noise_coef*noise_data
        

                
        # concatinate the noises and conditions
        generated_data = torch.cat([abs(noise_data), random_doc_tensor.to(device)],-1)
        
        
        # concatinate the noises and conditions
        generated_data = torch.cat([abs(torch.randn((len(data_for_generator), vec_size))).to(device), random_doc_tensor.to(device)],-1)
    
        # run diffusion model to generate new embedding
        for _ in range(int(n_steps)):
            # print(generated_data)
            noise_data = generated_data[:,:vec_size] + noise_factor * torch.randn_like(generated_data[:,:vec_size])
            input_vec = torch.cat([random_doc_tensor, noise_data],-1)
            generated_data = model(input_vec)
    
        res_list = generated_data.cpu().detach().numpy()
        generated_doc_by_diffusion = [res_list[i][:vec_size] for i in range(len(res_list))]
            
        # predict the tf_idf for new generated samples
        X_tfidf = np.random.rand(len(generated_doc_by_diffusion), len(generated_doc_by_diffusion[0]))
        
        for i in range(len(X_tfidf)):
            X_tfidf[i] = generated_doc_by_diffusion[i]
        
        generated_tf_idf_by_nn = model_tfidf.predict(X_tfidf)

        #remove duplicated generated data (the ones are very similar to each other) (maximum acceptable cosine similarity)
        generated_tf_idf_by_nn = remove_duplicate_embedding(generated_tf_idf_by_nn, 0.95)
        
        #normalize the values of tf_idf vector (nothing less than zero and nothing greater than one)
        normalized_generated_tf_idf = [tf_idf_normalizer(w) for w in generated_tf_idf_by_nn]
    
    
        #candidate documents in the ts
        candidate_docs_id = collect_ids[ts_ids]
        #filter candidates based on the keyword overlaps
        candidate_docs_tf_idf = [keyword_extractor(tf_idf_dict[w], list_of_words_in_tfidf, 100) for w in candidate_docs_id]
        candidate_scores = [keyword_similarity(w,storyline_keywords) for w in candidate_docs_tf_idf]
        candidates = [candidate_docs_id[i] for i in range(len(candidate_docs_id)) if candidate_scores[i] > min_keyword_overlapping]
    
        if len(candidates) < 1:
            #there is no document in the timestamp with the minimum acceptable keyword overlap
            continue
        # calculate the similarity between generated emb and seed document
        generated_sim_with_seed = [cosin_similarity_doc_emb(w, seed_embedding) for w in generated_doc_by_diffusion]
    
        #pick the best generated document
        picked_generated_emb = generated_doc_by_diffusion[generated_sim_with_seed.index(max(generated_sim_with_seed))]
        # pick the most similar candidate to the selected generated document
        candidate_vs_generated_cosine_similarity = [cosin_similarity_doc_emb(picked_generated_emb, contextual_dict[w]) for w in candidates]
        max_candidate_sim = max(candidate_vs_generated_cosine_similarity)
        
        if max_candidate_sim < min_acceptable_similarity:
            # the best candidate is still not similar to the previous document. going to the next ts
            continue
        seed_doc = candidates[candidate_vs_generated_cosine_similarity.index(max_candidate_sim)]
        # add the new pick into the soryline
        storyline.append(seed_doc)
        # update the storyline_keywords
        merge_keywords(storyline_keywords, keyword_extractor(tf_idf_dict[seed_doc], list_of_words_in_tfidf, 100), 0.9)
    return storyline


In [58]:
zx = story_gen_emb_tfidf_novel_method(3594, collect_ids) 
print(*[[w, title_id_dict[w]] for w in zx], sep='\n')

3/3 [==============================] - 0s 1ms/step
[3594, 'Biden Administration Plans to Offer Second Booster Shots to Those 50 and Up']
[3873, 'Coronavirus deaths in the U.S. fall to their lowest point since the summer.']
[7378, 'With cases rising, Mayor Eric Adams is keeping New York City’s preschool mask mandate.']
[8367, 'The BA.2 subvariant prompts a slight increase in new U.S. cases.']
[9608, 'Biden says Americans should decide for themselves if they want to wear masks on public transportation.']
[10651, 'The European Union says the emergency phase of the pandemic is over.']
[11054, 'F.D.A. sets aside June for potential decisions on vaccines for the youngest Americans.']
[20536, 'As known cases climb in southern Africa, testing on the continent has fallen off.']
[21862, 'Pfizer Says 3 Vaccine Doses Produce Strong Response in Young Children']
[22382, 'Jay Inslee, governor of Washington, tests positive for the virus.']
[27066, 'Youngest Children Could Get Covid Shots in Late June, 

In [59]:
# this function recieves a seed and generats the relevant story to that seed
def story_gen_just_emb_novel_method(seed_, collect_ids, min_size_generated_data_per_ts=50, min_acceptable_similarity= 0.9, min_keyword_overlapping= 0.60):
    ###SEARCH ALGORITHM
    starting_ts = ts_finder(seed_, collect_ids)
        
    seed_doc = seed_
    storyline = []
    storyline.append(seed_doc)
    # storyline_keywords = keyword_extractor(tf_idf_dict[seed_doc], list_of_words_in_tfidf, 100)
    # aux_doc = False
    for ts_ids in list(collect_ids.keys())[starting_ts+1:]:
        #preparing conditions and noises for diffusion model
        data_for_generator = []
        data_ids = [w[0] for w in data_timestamps[ts_ids]]
        seed_embedding = contextual_dict[seed_doc]
        temp = list()
        while len(data_for_generator) < min_size_generated_data_per_ts:
            for cent_ in centroids_ts_group[ts_ids]:
                temp[:vec_size] = seed_embedding
                temp[vec_size:] = cent_
                # data_for_generator.append([contextual_dict[seed_doc], cent_])
                data_for_generator.append(temp)
    
        random_doc_tensor = torch.randn((len(data_for_generator), len(data_for_generator[0]))).to(device)
        for i in range(len(data_for_generator)):
            random_doc_tensor[i] = torch.from_numpy(np.array(data_for_generator[i]))
                
        average_data = torch.mean(abs(random_doc_tensor))
        noise_data = torch.randn((len(data_for_generator), vec_size)).to(device)
        average_noise = torch.mean(abs(noise_data))
        noise_coef = average_data/average_noise
        noise_data = noise_coef*noise_data
        

                
        # concatinate the noises and conditions
        generated_data = torch.cat([abs(noise_data), random_doc_tensor.to(device)],-1)
        
        
        # concatinate the noises and conditions
        generated_data = torch.cat([abs(torch.randn((len(data_for_generator), vec_size))).to(device), random_doc_tensor.to(device)],-1)
    
        # run diffusion model to generate new embedding
        for _ in range(int(n_steps)):
            # print(generated_data)
            noise_data = generated_data[:,:vec_size] + noise_factor * torch.randn_like(generated_data[:,:vec_size])
            input_vec = torch.cat([random_doc_tensor, noise_data],-1)
            generated_data = model(input_vec)
    
        res_list = generated_data.cpu().detach().numpy()
        generated_doc_by_diffusion = [res_list[i][:vec_size] for i in range(len(res_list))]    
    
        #candidate documents in the ts
        candidate_docs_id = collect_ids[ts_ids]
        #filter candidates based on the keyword overlaps
        # candidate_docs_tf_idf = [keyword_extractor(tf_idf_dict[w], list_of_words_in_tfidf, 100) for w in candidate_docs_id]
        # candidate_scores = [keyword_similarity(w,storyline_keywords) for w in candidate_docs_tf_idf]
        candidates = [candidate_docs_id[i] for i in range(len(candidate_docs_id))]
        # if len(candidates) < 1:
        #     #there is no document in the timestamp with the minimum acceptable keyword overlap
        #     continue
        # calculate the similarity between generated emb and seed document
        generated_sim_with_seed = [cosin_similarity_doc_emb(w, seed_embedding) for w in generated_doc_by_diffusion]
        #pick the best generated document
        picked_generated_emb = generated_doc_by_diffusion[generated_sim_with_seed.index(max(generated_sim_with_seed))]

        # pick the most similar candidate to the selected generated document
        candidate_vs_generated_cosine_similarity = [cosin_similarity_doc_emb(picked_generated_emb, contextual_dict[w]) for w in candidates]
        max_candidate_sim = max(candidate_vs_generated_cosine_similarity)
        
        if max_candidate_sim < min_acceptable_similarity:
            # the best candidate is still not similar to the previous document. going to the next ts
            continue
        seed_doc = candidates[candidate_vs_generated_cosine_similarity.index(max_candidate_sim)]
        # add the new pick into the soryline
        storyline.append(seed_doc)
        # update the storyline_keywords
        # merge_keywords(storyline_keywords, keyword_extractor(tf_idf_dict[seed_doc], list_of_words_in_tfidf, 100), 0.9)
    return storyline


In [60]:
zx_1 = story_gen_just_emb_novel_method(3594, collect_ids, min_acceptable_similarity= 0.94) 
print(*[[w, title_id_dict[w]] for w in zx_1], sep='\n')

[3594, 'Biden Administration Plans to Offer Second Booster Shots to Those 50 and Up']
[9219, '‘Thank You for Not Killing Us’']
[9413, 'Russian attack on Lviv shakes sense of security in a relatively safe city.']
[10291, 'Covid Outbreak in Beijing Prompts Order for Nearly Citywide Testing']
[10777, 'In El Salvador, the President Cracks Down on Civil Liberties, and Is Beloved for It']
[19731, 'Worries about the strain on health care grow in the U.S. as hot spots spread and hospitalizations rise.']
[21147, 'With Plunging Enrollment, a ‘Seismic Hit’ to Public Schools']
[27692, 'Moderna says new trial results show that a revised vaccine works better against Omicron.']
[30439, 'F.D.A. May Move Toward Updating Vaccines']


In [61]:
# this function recieves a seed and generats the relevant story to that seed
def story_gen_just_tfidf_novel_method(seed_, collect_ids, min_size_generated_data_per_ts=50, min_acceptable_similarity= 0.9, min_keyword_overlapping= 0.60):
    ###SEARCH ALGORITHM
    starting_ts = ts_finder(seed_, collect_ids)
        
    seed_doc = seed_
    storyline = []
    storyline.append(seed_doc)
    storyline_keywords = keyword_extractor(tf_idf_dict[seed_doc], list_of_words_in_tfidf, 100)
    aux_doc = False
    for ts_ids in list(collect_ids.keys())[starting_ts+1:]:
        #candidate documents in the ts
        candidate_docs_id = collect_ids[ts_ids]
        #filter candidates based on the keyword overlaps
        candidate_docs_tf_idf = [keyword_extractor(tf_idf_dict[w], list_of_words_in_tfidf, 100) for w in candidate_docs_id]
        candidate_scores = [keyword_similarity(w,storyline_keywords) for w in candidate_docs_tf_idf]
        temp = dict()
        for i in range(len(candidate_docs_id)):
            temp[candidate_scores[i]] = candidate_docs_id[i]
        max_val = max(list(temp.keys()))
        if max_val > min_keyword_overlapping:
            seed_doc = temp[max_val]
            # add the new pick into the soryline
            storyline.append(seed_doc)
            # update the storyline_keywords
            merge_keywords(storyline_keywords, keyword_extractor(tf_idf_dict[seed_doc], list_of_words_in_tfidf, 100), 0.9)
        else:
            continue

    return storyline


In [62]:
zx_2 = story_gen_just_tfidf_novel_method(3594, collect_ids, min_acceptable_similarity= 0.94, min_keyword_overlapping= 0.6) 
print(*[[w, title_id_dict[w]] for w in zx_2], sep='\n')

[3594, 'Biden Administration Plans to Offer Second Booster Shots to Those 50 and Up']
[4120, 'Biden gets his second booster after his Covid remarks.']
[7434, 'President Biden asks for more funding and Hong Kong’s death toll strains mortuaries: The week in Covid news.']
[7829, 'The C.D.C. director says she ‘really would encourage’ second boosters for older people and many with chronic conditions.']
[8009, 'F.D.A. Panel Explores Challenges of Revamping Coronavirus Vaccines']
[8584, 'Virus outbreaks are pushing some U.S. universities to reinstate mask mandates.']
[8965, 'Pfizer Says Booster Strengthens Immune Response for Children 5 to 11']
[9361, 'Americans Over 60 Should Get Second Booster, Official Says']
[9953, 'The U.S. is extending a vaccine rule for international travelers at its land borders.']
[10512, 'Vaccines for young children are being delayed by incomplete data, a top F.D.A. official suggests.']
[11054, 'F.D.A. sets aside June for potential decisions on vaccines for the youn

In [63]:
def distributed_similarity(storyline_, storyline_keywords_, candidate_doc_, diffused_doc_, tf_idf_dict_=tf_idf_dict, contextual_dict_=contextual_dict, weight_for_past_doc=0.9):
    """
    This function receives a storyline_ (a list of already picked stories), a storyline_keywords_(a sorted list of most important keywords in the story)
    a diffused_doc_ (a generated embedding by diffusion as a hypothesis candidate), candidate_doc_ as a document that we want to pick as the next one,  and weight_for_past_doc as a weight to control the contribution of past
    """
    weights_list = []
    for doc_ in storyline_:
        key_word_sim = keyword_extractor(tf_idf_dict_[doc_], list_of_words_in_tfidf, 100)
        weights_list.append(keyword_similarity(key_word_sim, storyline_keywords_))
    #normalize weights
    min_ = min(weights_list)
    max_ = max(weights_list)
    normalized_weights = [(w - min_)/(abs(max_ - min_) + 0.0001) for w in weights_list]
    sim_ = cosine_similarity([contextual_dict_[candidate_doc_]],[diffused_doc_])[0][0]
    w_f_p_d = [weight_for_past_doc**(t+1) for t in reversed(range(len(storyline_)))]
    for ind in range(len(storyline_)):
        sim_ += w_f_p_d[ind]*weights_list[ind]*cosine_similarity([contextual_dict_[storyline_[ind]]], [diffused_doc_])[0][0]
    #normalize cosine similarities
    sim_ = sim_/(sum(w_f_p_d)+1)
    return sim_

In [64]:
# this function recieves a seed and generats the relevant story to that seed
def story_gen_emb_tfidf_novel_method_w_distributed_attention(seed_, collect_ids, min_size_generated_data_per_ts=50, min_acceptable_similarity= 0.9, min_keyword_overlapping= 0.60):
    ###SEARCH ALGORITHM
    starting_ts = ts_finder(seed_, collect_ids)
        
    seed_doc = seed_
    storyline = []
    storyline.append(seed_doc)
    storyline_keywords = keyword_extractor(tf_idf_dict[seed_doc], list_of_words_in_tfidf, 100)
    
    for ts_ids in list(collect_ids.keys())[starting_ts+1:]:
        #preparing conditions and noises for diffusion model
        data_for_generator = []
        data_ids = [w[0] for w in data_timestamps[ts_ids]]
        seed_embedding = contextual_dict[seed_doc]
        temp = list()
        while len(data_for_generator) < min_size_generated_data_per_ts:
            for cent_ in centroids_ts_group[ts_ids]:
                temp[:vec_size] = seed_embedding
                temp[vec_size:] = cent_
                # data_for_generator.append([contextual_dict[seed_doc], cent_])
                data_for_generator.append(temp)
    
        random_doc_tensor = torch.randn((len(data_for_generator), len(data_for_generator[0]))).to(device)
        for i in range(len(data_for_generator)):
            random_doc_tensor[i] = torch.from_numpy(np.array(data_for_generator[i]))
                
        average_data = torch.mean(abs(random_doc_tensor))
        noise_data = torch.randn((len(data_for_generator), vec_size)).to(device)
        average_noise = torch.mean(abs(noise_data))
        noise_coef = average_data/average_noise
        noise_data = noise_coef*noise_data
        

                
        # concatinate the noises and conditions
        generated_data = torch.cat([abs(noise_data), random_doc_tensor.to(device)],-1)
        
        
        # concatinate the noises and conditions
        generated_data = torch.cat([abs(torch.randn((len(data_for_generator), vec_size))).to(device), random_doc_tensor.to(device)],-1)
    
        # run diffusion model to generate new embedding
        for _ in range(int(n_steps)):
            # print(generated_data)
            noise_data = generated_data[:,:vec_size] + noise_factor * torch.randn_like(generated_data[:,:vec_size])
            input_vec = torch.cat([random_doc_tensor, noise_data],-1)
            generated_data = model(input_vec)
    
        res_list = generated_data.cpu().detach().numpy()
        generated_doc_by_diffusion = [res_list[i][:vec_size] for i in range(len(res_list))]
            
        # predict the tf_idf for new generated samples
        X_tfidf = np.random.rand(len(generated_doc_by_diffusion), len(generated_doc_by_diffusion[0]))
        
        for i in range(len(X_tfidf)):
            X_tfidf[i] = generated_doc_by_diffusion[i]
        
        generated_tf_idf_by_nn = model_tfidf.predict(X_tfidf)

        #remove duplicated generated data (the ones are very similar to each other) (maximum acceptable cosine similarity)
        generated_tf_idf_by_nn = remove_duplicate_embedding(generated_tf_idf_by_nn, 0.95)
        
        #normalize the values of tf_idf vector (nothing less than zero and nothing greater than one)
        normalized_generated_tf_idf = [tf_idf_normalizer(w) for w in generated_tf_idf_by_nn]
    
    
        #candidate documents in the ts
        candidate_docs_id = collect_ids[ts_ids]
        #filter candidates based on the keyword overlaps
        candidate_docs_tf_idf = [keyword_extractor(tf_idf_dict[w], list_of_words_in_tfidf, 100) for w in candidate_docs_id]
        candidate_scores = [keyword_similarity(w,storyline_keywords) for w in candidate_docs_tf_idf]
        candidates = [candidate_docs_id[i] for i in range(len(candidate_docs_id)) if candidate_scores[i] > min_keyword_overlapping]
    
        if len(candidates) < 1:
            #there is no document in the timestamp with the minimum acceptable keyword overlap
            continue
        # calculate the similarity between generated emb and seed document
        generated_sim_with_seed = [cosin_similarity_doc_emb(w, seed_embedding) for w in generated_doc_by_diffusion]
    
        #pick the best generated document
        picked_generated_emb = generated_doc_by_diffusion[generated_sim_with_seed.index(max(generated_sim_with_seed))]
        # pick the most similar candidate to the selected generated document

        #using distributed attention similarity
        candidate_vs_generated_cosine_similarity = [distributed_similarity(storyline, storyline_keywords, w, picked_generated_emb) for w in candidates]


        
        # candidate_vs_generated_cosine_similarity = [cosin_similarity_doc_emb(picked_generated_emb, contextual_dict[w]) for w in candidates]
        max_candidate_sim = max(candidate_vs_generated_cosine_similarity)
        
        if max_candidate_sim < min_acceptable_similarity:
            # the best candidate is still not similar to the previous document. going to the next ts
            continue
        seed_doc = candidates[candidate_vs_generated_cosine_similarity.index(max_candidate_sim)]
        # add the new pick into the soryline
        storyline.append(seed_doc)
        # update the storyline_keywords
        merge_keywords(storyline_keywords, keyword_extractor(tf_idf_dict[seed_doc], list_of_words_in_tfidf, 100), 0.9)
    return storyline


In [65]:
zx_3 = story_gen_emb_tfidf_novel_method_w_distributed_attention(3594, collect_ids, min_acceptable_similarity= 0.94, min_keyword_overlapping= 0.6) 
print(*[[w, title_id_dict[w]] for w in zx_3], sep='\n')

3/3 [==============================] - 0s 860us/step
[3594, 'Biden Administration Plans to Offer Second Booster Shots to Those 50 and Up']
[3873, 'Coronavirus deaths in the U.S. fall to their lowest point since the summer.']
[7378, 'With cases rising, Mayor Eric Adams is keeping New York City’s preschool mask mandate.']
[7976, 'European health agencies say it is ‘too early’ for second boosters for most people.']
[8367, 'The BA.2 subvariant prompts a slight increase in new U.S. cases.']
[8584, 'Virus outbreaks are pushing some U.S. universities to reinstate mask mandates.']
[8965, 'Pfizer Says Booster Strengthens Immune Response for Children 5 to 11']
[9361, 'Americans Over 60 Should Get Second Booster, Official Says']
[9608, 'Biden says Americans should decide for themselves if they want to wear masks on public transportation.']
[10651, 'The European Union says the emergency phase of the pandemic is over.']
[11054, 'F.D.A. sets aside June for potential decisions on vaccines for the you

In [83]:
all_ids = []
for i in collect_ids:
    for j in collect_ids[i]:
        all_ids.append(j)

In [84]:
seen_doc = set()
result_dict_story_gen_emb_tfidf_novel_method_w_distributed_attention = dict()
for se in tqdm(all_ids):
    if se in seen_doc:
        continue
    print(se)
    res = story_gen_emb_tfidf_novel_method_w_distributed_attention(se, collect_ids, min_acceptable_similarity= 0.94, min_keyword_overlapping= 0.6) 
    result_dict_story_gen_emb_tfidf_novel_method_w_distributed_attention[se] = res
    for d in res:
        seen_doc.add(d)

  0%|          | 0/8322 [00:00<?, ?it/s]

22989
3/3 [==============================] - 0s 1ms/step


  0%|          | 1/8322 [03:05<429:55:02, 186.00s/it]

23079
3/3 [==============================] - 0s 892us/step


  0%|          | 2/8322 [06:40<469:03:04, 202.95s/it]

...

12281
3/3 [==============================] - 0s 926us/step


 12%|█▏        | 1032/8322 [25:41:25<142:21:28, 70.30s/it]

11963
3/3 [==============================] - 0s 914us/step


 12%|█▏        | 1039/8322 [25:42:05<180:09:30, 89.05s/it]


KeyboardInterrupt: 

In [85]:
# pickle.dump(result_dict_story_gen_emb_tfidf_novel_method_w_distributed_attention, open('25_result_dict_story_gen_emb_tfidf_novel_method_w_distributed_attention_v128.p', 'wb'))

In [86]:
# seed_documents_for_experiments = list(result_dict_story_gen_emb_tfidf_novel_method_w_distributed_attention.keys())

In [93]:
# pickle.dump(title_id_dict, open('25_title_id_dict_v128.p', 'wb'))
# pickle.dump(abstract_id_dict, open('25_abstract_id_dict_v128.p', 'wb'))
# pickle.dump(text_id_dict, open('25_text_id_dict_v128.p', 'wb'))
# pickle.dump(date_id_dict, open('25_date_id_dict_v128.p', 'wb'))
# pickle.dump(tf_idf_dict, open('25_tf_idf_dict_v128.p', 'wb'))
# pickle.dump(data_timestamps, open('25_data_timestamps_v128.p', 'wb'))
# pickle.dump(contextual_doc_embedding, open('25_contextual_doc_embedding_v128.p', 'wb'))
# # pickle.dump(contextual_bert_embedding, open('25_contextual_bert_embedding_v128.p', 'wb'))
# pickle.dump(contextual_dict, open('25_contextual_dict_v128.p','wb'))
# pickle.dump(list_of_words_in_tfidf, open('25_list_of_words_in_tfidf_v128.p', 'wb'))
# pickle.dump(seed_documents_for_experiments, open('25_seed_documents_for_experiments_v128.p', 'wb'))

In [91]:
# keyword based model
seen_doc = set()
result_dict_story_gen_just_tfidf_novel_method = dict()
for se in tqdm(seed_documents_for_experiments):
    if se in seen_doc:
        continue
    print(se)
    res = story_gen_just_tfidf_novel_method(se, collect_ids, min_acceptable_similarity= 0.94, min_keyword_overlapping= 0.6) 
    result_dict_story_gen_just_tfidf_novel_method[se] = res
    for d in res:
        seen_doc.add(d)

  0%|          | 0/471 [00:00<?, ?it/s]

22989


  0%|          | 1/471 [00:59<7:44:01, 59.24s/it]

23079


  0%|          | 2/471 [01:58<7:42:21, 59.15s/it]

...

 99%|█████████▉| 467/471 [6:15:45<03:20, 50.16s/it]

12029


100%|█████████▉| 469/471 [6:16:35<01:17, 38.54s/it]

12074


100%|██████████| 471/471 [6:17:25<00:00, 48.08s/it]


In [92]:
pickle.dump(result_dict_story_gen_just_tfidf_novel_method, open('25_rresult_dict_story_gen_just_tfidf_novel_method_v128.p', 'wb'))

In [95]:
# embedding based model
seen_doc = set()
result_dict_story_gen_just_emb_novel_method = dict()
for se in tqdm(seed_documents_for_experiments):
    if se in seen_doc:
        continue
    print(se)
    res = story_gen_just_emb_novel_method(se, collect_ids, min_acceptable_similarity= 0.94) 
    result_dict_story_gen_just_emb_novel_method[se] = res
    for d in res:
        seen_doc.add(d)

  0%|          | 0/471 [00:00<?, ?it/s]

22989


  0%|          | 1/471 [00:06<47:18,  6.04s/it]

23079


  0%|          | 2/471 [00:12<47:09,  6.03s/it]

...

 99%|█████████▉| 467/471 [40:23<00:20,  5.25s/it]

12253


 99%|█████████▉| 468/471 [40:28<00:15,  5.25s/it]

12281


100%|██████████| 471/471 [40:33<00:00,  5.17s/it]


In [96]:
pickle.dump(result_dict_story_gen_just_emb_novel_method, open('25_result_dict_story_gen_just_emb_novel_method_v128.p', 'wb'))

In [ ]:
# the proposed model without distributed similarity mechanism
seen_doc = set()
result_dict_story_gen_emb_tfidf_novel_method = dict()
for se in tqdm(seed_documents_for_experiments):
    if se in seen_doc:
        continue
    print(se)
    res = story_gen_emb_tfidf_novel_method(se, collect_ids)
    result_dict_story_gen_emb_tfidf_novel_method[se] = res
    for d in res:
        seen_doc.add(d)

  0%|          | 0/471 [00:00<?, ?it/s]

22989
3/3 [==============================] - 0s 993us/step


  0%|          | 1/471 [02:30<19:36:42, 150.22s/it]

23079
3/3 [==============================] - 0s 889us/step


  0%|          | 2/471 [04:55<19:10:19, 147.16s/it]

...

23020
3/3 [==============================] - 0s 931us/step


  5%|▌         | 24/471 [1:08:19<22:15:51, 179.31s/it]

23163
3/3 [==============================] - 0s 916us/step


  5%|▌         | 25/471 [1:10:40<20:47:03, 167.77s/it]

23177
3/3 [==============================] - 0s 910us/step


In [ ]:
pickle.dump(result_dict_story_gen_emb_tfidf_novel_method, open('25_result_dict_story_gen_emb_tfidf_novel_method_v128.p', 'wb'))